In [ ]:
%pip install pymongo
%pip install python-dotenv

Importación de librerias

In [3]:
import pymongo

import os
from dotenv import load_dotenv

Carga los secretos del archivo **.env**

In [4]:
# Carga los secretos del archivo .env
load_dotenv()

# Accede a los valores definidos en .env
MONGODB_SERVER = os.getenv("MONGODB_SERVER")

print(MONGODB_SERVER[:10] + '*****' + MONGODB_SERVER[20:])

mongodb://*****27017/


### Conexión MongoDB

In [5]:
mongoClient = pymongo.MongoClient(MONGODB_SERVER, serverSelectionTimeoutMS=5000)

try:
    print(mongoClient.server_info())
except Exception:
    print("Unable to connect to the server.")

{'version': '5.0.3', 'gitVersion': '657fea5a61a74d7a79df7aff8e4bcf0bc742b748', 'targetMinOS': 'Windows 7/Windows Server 2008 R2', 'modules': [], 'allocator': 'tcmalloc', 'javascriptEngine': 'mozjs', 'sysInfo': 'deprecated', 'versionArray': [5, 0, 3, 0], 'openssl': {'running': 'Windows SChannel'}, 'buildEnvironment': {'distmod': 'windows', 'distarch': 'x86_64', 'cc': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.26.28806 for x64', 'ccflags': '/nologo /WX /FImongo/platform/basic.h /EHsc /W3 /wd4068 /wd4244 /wd4267 /wd4290 /wd4351 /wd4355 /wd4373 /wd4800 /wd4251 /wd4291 /we4013 /we4099 /we4930 /errorReport:none /MD /O2 /Oy- /bigobj /utf-8 /permissive- /Zc:__cplusplus /Zc:sizedDealloc /volatile:iso /diagnostics:caret /std:c++17 /Gw /Gy /Zc:inline', 'cxx': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.26.28806 for x64', 'cxxflags': '/TP', 'linkflags': '/nologo /DEBUG /INCREMENTAL:NO /LARGEADDRESSAWARE /OPT:REF', 'target_arch': 'x86_64', 'target_os': 'windows', 'cppdefine

Definición BBDD, colecciones e indices en MongoDB

In [6]:
mongoDb = mongoClient['web3_tfm']
collection_tx = mongoDb['account_transactions']

In [6]:
# Eliminar el atributo "input" de todas las transacciones
# El almacenamiento de este campo multiplica el espacio requerido para los datos y no es de relevancia para lo que se está revisando
result = collection_tx.update_many({}, {"$unset": {"input": ""}})

# Imprimir el resultado
print(f"Se eliminaron '{result.modified_count}' ocurrencias del atributo 'input'.")

Se eliminaron '1511575' ocurrencias del atributo 'input'.


### Carga colección *daily_transactions_summary* (con funciones de agregación MongoDB)
Resumen del número de usuarios y transacciones realizadas diariamente entre dos redes

In [ ]:
# Ejecutar la consulta de agregación
pipeline = [
    {
        "$addFields": {
            "date": { "$toDate": "$date" },
            "from_network": { "$convert": { "input": "$from_network", "to": "int" }},
            "to_network": { "$convert": { "input": "$to_network", "to": "int" }}
        }
    },
    {
        "$group": {
            "_id": {
                "fecha": { "$dateToString": { "format": "%Y-%m-%d", "date": "$date" } },
                "to_address": "$to",
                "from_network": "$from_network",
                "to_network": "$to_network"
            },
            "daily_transactions": { "$sum": 1 },
            "from_address_distintos": { "$addToSet": "$from" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "date": "$_id.fecha",
            "to_address": "$_id.to_address",
            "from_network": "$_id.from_network",
            "to_network": "$_id.to_network",
            "daily_transactions": 1,
            "unique_from_address": { "$size": "$from_address_distintos" }
        }
    },
    { 
        "$out": "daily_transactions_summary"
    }
]

result = collection_tx.aggregate(pipeline)

### Carga colección *gas_price_history* (con funciones de agregación MongoDB)
Calcula la mediana del precio diario del gas para las transacciones de los puentes en la red Ethereum

In [30]:
pipeline = [
    {
        "$addFields": {
            "date": { "$substr": ["$date", 0, 10] }
        }
    },
    {
        "$group": {
            "_id": {
                "date": "$date",
                "from_network": "$from_network"
            },
            "gas_prices": { "$push": "$gasPrice" }
        }
    },
    {
        "$addFields": {
            "count": { "$size": "$gas_prices" },
            "sorted_gas_prices": {
                "$function": {
                    "body": """
                        function(arr) {
                            return arr.sort((a, b) => a - b);
                        }
                    """,
                    "args": ["$gas_prices"],
                    "lang": "js"
                }
            }
        }
    },
    {
        "$addFields": {
            "median_index": { "$divide": [{ "$subtract": ["$count", 1] }, 2] }
        }
    },
    {
        "$addFields": {
            "median_gas_price": {
                "$avg": [
                    { "$arrayElemAt": ["$sorted_gas_prices", { "$toInt": "$median_index" }] },
                    { "$arrayElemAt": ["$sorted_gas_prices", { "$add": [{ "$toInt": "$median_index" }, 1] }] }
                ]
            }
        }
    },
    {
        "$project": {
            "_id": 0,  # Eliminar el atributo _id
            "date": "$_id.date",
            "from_network": "$_id.from_network",
            "median_gas_price": 1
        }
    },
    { 
        "$out": "gas_price_history"
    }
]

result = collection_tx.aggregate(pipeline)


### Carga colección *daily_methods_summary* (con funciones de agregación MongoDB)
Resumen de las funciones utilizados en las transacciones

In [35]:
pipeline = [
  {
    "$addFields": {
      "date": { "$substr": ["$date", 0, 10] }
    }
  },
  {
    "$lookup": {
      "from": "methods",
      "localField": "methodId",
      "foreignField": "methodId",
      "as": "method_info"
    }
  },
  {
    "$group": {
      "_id": {
        "date": "$date",
        "from_network": "$from_network",
        "to_network": "$to_network",
        "methodId": "$methodId"
      },
      "count": { "$sum": 1 },
      "functionName": { "$first": { "$arrayElemAt": ["$method_info.functionName", 0] } }
    }
  },
  {
    "$project": {
      "_id": 0,
      "date": "$_id.date",
      "from_network": "$_id.from_network",
      "to_network": "$_id.to_network",
      "methodId": "$_id.methodId",
      "functionName": 1,
      "count": 1
    }
  },
  { 
    "$out": "daily_methods_summary"
  }
]

result = collection_tx.aggregate(pipeline, allowDiskUse=True)